<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_01_Cast_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee
All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the
LICENSE file in the root directory of this source tree. 

#Create Charts and store in MongoDB

#Setup

##Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [1]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
#!pip -qq install python_string_utils
#!pip -qq install python-docx                                #https://python-docx.readthedocs.io/en/latest/
#!pip install pyswisseph==2.10.2.0.dev1

# required for accessing Atlas servers 
!/usr/bin/python3 -m pip install "pymongo[srv]"
!apt install mongodb > /dev/null

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 4.0 MB/s 




In [8]:
!pip -qq install python-docx 

     |████████████████████████████████| 5.6 MB 4.1 MB/s 


In [9]:
from pymongo import MongoClient
import pandas as pd
import dateutil
#import string_utils
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [5]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py        # large number of utility functions
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py    # yog data - description and conditions 



In [10]:
# ---------------------------
import p21
import p21swe
import p21utils
import p21utilsR
import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21swe)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21YogInfo)

p21utilsR imported
p21swe imported
p21utils imported
p21utilsR imported
8 Yogs on record


<module 'p21YogInfo' from '/content/p21YogInfo.py'>

In [11]:
#Download Swiss Ephemeris Files 
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21swe.C01_configSWE()

##Read External Data
from CSV files


In [12]:
import pandas as pd

#!wget -O peopleData.csv -q https://raw.githubusercontent.com/prithwis/parashar21/main/data/Test5Data.txt # 5 Person Test Data
!wget -O peopleData.csv -q https://github.com/prithwis/parashar21/raw/main/data/Rodden39663.csv

#!gdown -O peopleData.csv https://drive.google.com/uc?id=1CVo1gnMZyCD0EZvAoF_wRdTwL7yqIT6L # 5 Person Test Data
#!gdown -O peopleData.csv https://drive.google.com/uc?id=1dlvqcygJJh0CfyY_X2MLrWm-W9U0z_6l # 39663 Rodden Data

#!sed -i '1d' peopleData.csv 
#!cat peopleData.csv

df = pd.read_csv('peopleData.csv')
df.head()

,Gender,Voc1_Cat,Voc1_Det,Voc2_Cat,Voc2_Det,Voc3_Cat,Voc3_Det,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_Off,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name
0,M,Entertain/Music,Instrumentalist (Guitarist),Entertain/Music,Song writer,Entertain/Music,"Vocalist/ Pop, Rock, etc.",6,10,1968,17:35,h1e,1.0,48.550000,3.300000,MET,standard,A_Dominique
1,M,Writers,Columnist/ journalist,Writers,Detective/ Mystery,Writers,Fiction (Novelist),19,12,1947,20:00,h1e,1.0,47.383333,0.683333,MET,standard,A_D_G_
2,M,Entertainment,Actor/ Actress,nil,nil,nil,nil,21,2,1911,03:00,h1e,1.0,59.916667,10.750000,MET,standard,Aabel_Andreas
3,M,Entertainment,Actor/ Actress,Entertainment,Voice-Over,nil,nil,25,4,1902,07:15,h1e,1.0,59.916667,10.750000,MET,standard,Aabel_Per
4,M,Entertainment,TV host/ Personality,Politics,Government employee (Minister),nil,nil,4,1,1933,12:30,h0w,-0.0,38.350000,-0.483333,GMT,standard,Aal_Pomares_Henri_Francois


#Cast & Store charts in JSON file
Driver Code

In [13]:
%%time
p21.ChartType = 'Rashi'                                 # one of ['Rashi','Navamsa']
p21utilsR.C61_Cast2JSON(df)

0 records processed
1000 records processed
2000 records processed
3000 records processed
4000 records processed
5000 records processed
6000 records processed
7000 records processed
8000 records processed
9000 records processed
10000 records processed
11000 records processed
12000 records processed
13000 records processed
14000 records processed
15000 records processed
16000 records processed
17000 records processed
18000 records processed
19000 records processed
20000 records processed
21000 records processed
22000 records processed
23000 records processed
24000 records processed
25000 records processed
26000 records processed
27000 records processed
28000 records processed
error on record  28803
29000 records processed
30000 records processed
31000 records processed
32000 records processed
33000 records processed
34000 records processed
35000 records processed
36000 records processed
37000 records processed
38000 records processed
39000 records processed
CPU times: user 1min 7s, sys: 

In [ ]:
%%time
p21.ChartType = 'Rashi'                                 # one of ['Rashi','Navamsa']

writeFile =  open("peopleData.json", "w")  
for person in range(len(df)):
    try :
        personData =df.iloc[person]
        p21swe.C02_parsePersonData(personData)                  # split the data, extract required data into fields
                                                                # create skeleton chart json doc as p21.chart
        p21swe.C03_convertDates()                               # convert date, time of birth into universal time, calculate Ayanamsha
        p21swe.C04_calculateGrahaPositions()                    # calculate the Lon of each Graha along with Retrograde staus

                    #Horoscope is defined in terms of Natal Longitudes and Retrogrades
                    # for example
                    #GLon = {"La":98.5,"Su":178.9,"Mo":250.6,"Ma":196.2,"Me":193.2,"Ju":274.8,"Ve":153.8,"Sa":270.2,"Ra":122.1,"Ke":302.1}
                    #GRet = {"La":False,"Su":False,"Mo":False,"Ma":False,"Me":True,"Ju":False,"Ve":False,"Sa":False,"Ra":True,"Ke":True}

        p21swe.C05_buildGLonGRet()                              # create two json docs ( python dicts) to store Graha Lon and Ret status
                                                                # these are stored as global variables p21.GLon, p21.GRet
        p21utils.appendDict(p21.chart,p21.GLonRet)              # add Lon, Ret information to chart
        
        # ----------------------------------------
        p21utils.R11_LocateGrahaInRashi()
        
        p21utils.C10_DetermineBhavs()                           # determine Houses for person
        p21utils.appendDict(p21.chart,p21.BhavNBhavA)           # add BhavN, BhavA information to chart   
         
        # ----------------------------------------    
        p21utils.C11_DetermineLord()                            # determine Lords of Bhavs
        p21utils.appendDict(p21.chart,p21.LordInfo)             #  add information on Lords   

        p21utils.C12_BhavOfGraha_Lord()
        p21utils.appendDict(p21.chart,p21.BhavOfGraha_LordInfo) #  add information on Bhav Residency of Graha and Lords  
        
        
        p21utils.C21_DeterminePositions()
        p21utils.appendDict(p21.chart,p21.Positions)            #  add information on Graha / Lord POSITIONS  

        p21utils.C31_DetermineAspects()
        p21utils.appendDict(p21.chart,p21.Aspects)              #  add information on Graha / Lord ASPECTS  

        p21utils.C41_DetermineConjuncts()           
        p21utils.appendDict(p21.chart,p21.Conjuncts)            #  add information on Graha / Lord Conjuncts
    except :
        print('error on record ', person)
    else :
        #insert_result = kollection.insert_one(p21.chart)        # insert one json doc, chart to database
        #insert_result.acknowledged                              # Confirms that insert is successful
                                                                 # Bypassing insertion into MongoDB will make processing much faster
        json.dump(p21.chart, writeFile)                          # Storing chart data as JSON file
    finally:
        if ( person % 1000 ) == 0:
            print(person, "records processed")
    
writeFile.close()
#!cp peopleData.json /content/drive/'My Drive'/Prithwis/Parashar21/ChartCast/ChartData2/

0 records processed
CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 11.6 ms


#Connect to MongoDB

In [15]:
from google.colab import drive
drive.mount('/content/drive')
# --------------------------------------------
# copy credentials file
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/parashar21URI.py credentials.py
!cp /content/drive/'My Drive'/Praxis/WebCredentials/Parashar21_Atlas_18Sep22.py credentials.py
# -----------------------------------------------

#from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI
from credentials import parashar21URI

# -----------------------------------------------
mongoURI = parashar21URI                                   # for remote server


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 

['Parashar21', 'admin', 'local']

In [23]:
#db = client.btncgfi3a5bxyea
db = client.Parashar21
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
pd.DataFrame(list(db.list_collections()))
# empty list '[]' means that there are not collections in database

,name,type,options,info,idIndex
0,khona21,collection,{},"{'readOnly': False, 'uuid': b'\xdc\xdeO1&sBU\x...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"


In [ ]:
#db = client.btncgfi3a5bxyea

#db.khona21Test.drop()                   # Test DB
#db.khona21.drop()                      # Main DB do not drop, accidentally

#db.create_collection('khona21Test')      # https://feminisminindia.com/2019/06/07/khawna-khana-poet-rural-bengal/
                                        # https://en.wikipedia.org/wiki/Khana_(poet)
#kollection = db.khona21Test                # Set the collection to work with

#Check and record Initial DB status
#countInitial = kollection.count_documents({})
#dbSizeInitial = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
#print(round(dbSizeInitial),'KB database with ', countInitial, ' docs in current collection' )

#Load into Mongo

In [22]:
%%time
#!mongoimport --host "{rhostName}" --port "27017" --db "{rdbName}" --collection "khona21Test"  --drop --file "peopleData.json" -u "{ruserName}" -p "{rpassWord}"
#!mongoimport --uri "{rMongoURI}"  --collection "restaurants"  --drop --file "restaurants.json" 
#the original mongoURI needs to be changed to reflect the database where the collection will be loaded
mongoURIload = mongoURI.replace('?retryWrites=true&w=majority','Parashar21?')
##!mongoimport --uri "{rMongoURI}"  --collection "khona21Test"  --drop --file "peopleData.json" 
!mongoimport --uri "{mongoURIload}"  --collection "khona21"  --drop --file "peopleData.json" 

2022-09-21T05:14:46.874+0000	connected to: localhost
2022-09-21T05:14:47.095+0000	dropping: Parashar21.khona21
2022-09-21T05:14:48.235+0000	[........................] Parashar21.khona21	3.53MB/137MB (2.6%)
2022-09-21T05:14:51.237+0000	[........................] Parashar21.khona21	5.40MB/137MB (3.9%)
2022-09-21T05:14:54.245+0000	[##......................] Parashar21.khona21	13.9MB/137MB (10.2%)
2022-09-21T05:14:57.235+0000	[###.....................] Parashar21.khona21	20.8MB/137MB (15.2%)
2022-09-21T05:15:00.235+0000	[#####...................] Parashar21.khona21	31.1MB/137MB (22.8%)
2022-09-21T05:15:03.235+0000	[######..................] Parashar21.khona21	38.0MB/137MB (27.8%)
2022-09-21T05:15:06.235+0000	[#######.................] Parashar21.khona21	44.9MB/137MB (32.9%)
2022-09-21T05:15:09.235+0000	[#########...............] Parashar21.khona21	55.2MB/137MB (40.4%)
2022-09-21T05:15:12.235+0000	[##########..............] Parashar21.khona21	62.1MB/137MB (45.5%)
2022-09-21T05:15:15.235+000

#Check Database Status

In [24]:
kollection = db.khona21
#kollection = db.khona21Test
countFinal = kollection.count_documents({})
dbSizeFinal = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
#DocsInserted = countFinal - countInitial
#AddSpaceUsed = round(dbSizeFinal - dbSizeInitial)
#if DocsInserted > 0:
#    AvgDocSize = round(AddSpaceUsed/DocsInserted)
#else:
#    AvgDocSize = 'na'
#print(DocsInserted,' docs inserted, ',AddSpaceUsed,' KB added , with avg doc size = ', AvgDocSize)
print(round(dbSizeFinal),'KB database with ', countFinal, ' docs in current collection' )

146582 KB database with  39662  docs in current collection


In [25]:
# Check data loaded into MongoDb
#c = kollection.find()
#for crnt in c:
#    print(crnt)
pd.DataFrame(list(kollection.find({},{'_id':0}).limit(10)))

,pid,GLon,GRet,BhavN,BhavA,Lord,LordRashiN,LordRashiA,GrahaLordBhav,GRashiN,...,inEnemyL,GAspects2,GAspectedBy2,BAspectedBy2,BAspectedByBL2,GConjunctsG2,BLConjunctsG2,BLConjunctsBL2,beneficG,maleficG
0,"{'tags': ['Entertain/Music', 'Entertain/Music'...","{'La': 330.585, 'Su': 170.058, 'Mo': 352.364, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[ , Meen, Mesh, Vrish, Mithun, Karkat, Simha, ...","[ , Ju, Ma, Ve, Me, Mo, Su, Me, Ve, Ma, Ju, Sa...","[ , 5, 5, 7, 7, 12, 6, 7, 7, 5, 5, 12, 12]","[ , Simha, Simha, Tula, Tula, Meen, Kanya, Tul...","{'Su': [6], 'Mo': [5], 'Ma': [2, 9], 'Me': [4,...","{'La': 12, 'Su': 6, 'Mo': 12, 'Ma': 5, 'Me': 7...",...,"[False, False, False, False, False, False, Fal...","{'La': ['Su', 'Ke'], 'Su': ['Sa', 'Ra', 'Mo', ...","{'Su': ['Sa', 'Ra', 'Mo', 'La'], 'Ke': ['Sa', ...","{'7': ['Sa', 'Ra', 'Mo', 'La'], '1': ['Su', 'M...","{'1': ['9', '6', '2'], '2': ['4', '1', '8', '3...","{'Su': ['Ke'], 'Ke': ['Su'], 'Mo': ['Sa', 'Ra'...","{'1': ['Ma'], '2': ['Ju'], '3': ['Me'], '4': [...","{'1': ['9', '2'], '2': ['10', '1'], '3': ['4',...","{'Su': False, 'Mo': True, 'Ma': True, 'Me': Fa...","{'Su': True, 'Mo': False, 'Ma': False, 'Me': F..."
1,"{'tags': ['Writers', 'Writers', 'Writers'], 'c...","{'La': 100.281, 'Su': 243.912, 'Mo': 323.344, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]","[ , Karkat, Simha, Kanya, Tula, Vrishchik, Dha...","[ , Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma, Ve...","[ , 11, 9, 8, 10, 5, 8, 4, 4, 8, 5, 10, 8]","[ , Kumbh, Dhanu, Vrishchik, Makar, Simha, Vri...","{'Su': [2], 'Mo': [1], 'Ma': [5, 10], 'Me': [3...","{'La': 4, 'Su': 9, 'Mo': 11, 'Ma': 5, 'Me': 8,...",...,"[False, False, False, False, False, False, Fal...","{'La': ['Ve'], 'Mo': ['Ma'], 'Ma': ['Me', 'Mo'...","{'Ve': ['Sa', 'La'], 'Ma': ['Mo'], 'Mo': ['Ma'...","{'7': ['Sa', 'La'], '12': ['Su'], '2': ['Mo'],...","{'1': ['11', '4', '6', '9'], '2': ['1'], '3': ...","{'Me': ['Ju'], 'Ju': ['Me']}","{'3': ['Ju'], '6': ['Me'], '9': ['Me'], '12': ...","{'3': ['9', '6'], '6': ['3', '12'], '9': ['3',...","{'Su': False, 'Mo': True, 'Ma': True, 'Me': Fa...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ..."
2,"{'tags': ['Entertainment', 'nil', 'nil'], 'ck'...","{'La': 219.599, 'Su': 308.713, 'Mo': 217.934, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7]","[ , Vrishchik, Dhanu, Makar, Kumbh, Meen, Mesh...","[ , Ma, Ju, Sa, Sa, Ju, Ma, Ve, Me, Mo, Su, Me...","[ , 9, 7, 1, 1, 7, 9, 11, 10, 8, 11, 10, 11]","[ , Dhanu, Tula, Mesh, Mesh, Tula, Dhanu, Kumb...","{'Su': [10], 'Mo': [9], 'Ma': [1, 6], 'Me': [8...","{'La': 8, 'Su': 11, 'Mo': 8, 'Ma': 9, 'Me': 10...",...,"[False, False, True, False, False, True, False...","{'Ju': ['Su', 'Ra', 'Ve', 'Sa'], 'Sa': ['Me', ...","{'Su': ['Ju'], 'Ve': ['Ju'], 'Sa': ['Ke', 'Ju'...","{'7': ['Mo', 'La'], '10': ['Su', 'Ve'], '5': [...","{'3': ['4', '3'], '4': ['5', '2'], '5': ['6', ...","{'Su': ['Ve'], 'Ve': ['Su'], 'Ju': ['Ke'], 'Ke...","{'2': ['Ke'], '3': ['Ra'], '4': ['Ra'], '5': [...","{'7': ['10'], '10': ['7', '12'], '12': ['10']}","{'Su': True, 'Mo': True, 'Ma': True, 'Me': Fal...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ..."
3,"{'tags': ['Entertainment', 'Entertainment', 'n...","{'La': 72.155, 'Su': 11.613, 'Mo': 218.817, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]","[ , Mithun, Karkat, Simha, Kanya, Tula, Vrishc...","[ , Me, Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma...","[ , 1, 8, 1, 1, 11, 1, 10, 10, 10, 10, 1, 11]","[ , Mesh, Vrishchik, Mesh, Mesh, Kumbh, Mesh, ...","{'Su': [3], 'Mo': [2], 'Ma': [6, 11], 'Me': [1...","{'La': 3, 'Su': 1, 'Mo': 8, 'Ma': 1, 'Me': 1, ...",...,"[False, False, False, False, False, False, Fal...","{'Su': ['Ra'], 'Ma': ['Ra', 'Mo'], 'Me': ['Ra'...","{'Ra': ['Su', 'Me', 'Ma', 'Sa'], 'Mo': ['Ma'],...","{'7': ['La'], '5': ['Ke', 'Su', 'Me', 'Sa', 'M...","{'

In [ ]:
#Retrieves Charts based on tags
pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})))
c = kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})
for crnt in c:
    print(crnt)

{'pid': {'tags': ['Author', 'Homemaker', 'PostGraduate']}, 'GLon': {'La': 310.415, 'Su': 243.701, 'Mo': 201.001, 'Ma': 273.467, 'Me': 222.269, 'Ju': 62.748, 'Ve': 284.966, 'Sa': 318.17, 'Ra': 41.248, 'Ke': 221.248}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': False, 'Ju': True, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
{'pid': {'tags': ['Biologist', 'Scientist', 'PostGraduate']}, 'GLon': {'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'Ma': 41.525, 'Me': 138.242, 'Ju': 101.216, 'Ve': 131.788, 'Sa': 265.084, 'Ra': 282.763, 'Ke': 102.763}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': True, 'Ra': False, 'Ke': False}}


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)